In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
from torch.utils.tensorboard import SummaryWriter

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs, CovidDataset, Rescale, ToTensor
from utils import averageModels

In [2]:
writer = SummaryWriter()

In [3]:
dataset = CovidDataset(csv_path='./train.csv', transform=transforms.Compose([Rescale(64), ToTensor()]))
print(type(dataset))

<class 'FLDataset.CovidDataset'>


In [4]:
print(len(dataset))

3012


In [5]:
for i in range(len(dataset)):
    item = dataset.__getitem__(i)
    print("item ", i , " image shape= ", item['image'].shape)

item  0  image shape=  torch.Size([1, 64, 64])
item  1  image shape=  torch.Size([1, 64, 64])
item  2  image shape=  torch.Size([1, 64, 64])
item  3  image shape=  torch.Size([1, 64, 64])
item  4  image shape=  torch.Size([1, 64, 64])
item  5  image shape=  torch.Size([1, 64, 64])
item  6  image shape=  torch.Size([1, 64, 64])
item  7  image shape=  torch.Size([1, 64, 64])
item  8  image shape=  torch.Size([1, 64, 64])
item  9  image shape=  torch.Size([1, 64, 64])
item  10  image shape=  torch.Size([1, 64, 64])
item  11  image shape=  torch.Size([1, 64, 64])
item  12  image shape=  torch.Size([1, 64, 64])
item  13  image shape=  torch.Size([1, 64, 64])
item  14  image shape=  torch.Size([1, 64, 64])
item  15  image shape=  torch.Size([1, 64, 64])
item  16  image shape=  torch.Size([1, 64, 64])
item  17  image shape=  torch.Size([1, 64, 64])
item  18  image shape=  torch.Size([1, 64, 64])
item  19  image shape=  torch.Size([1, 64, 64])
item  20  image shape=  torch.Size([1, 64, 64])
it

item  175  image shape=  torch.Size([1, 64, 64])
item  176  image shape=  torch.Size([1, 64, 64])
item  177  image shape=  torch.Size([1, 64, 64])
item  178  image shape=  torch.Size([1, 64, 64])
item  179  image shape=  torch.Size([1, 64, 64])
item  180  image shape=  torch.Size([1, 64, 64])
item  181  image shape=  torch.Size([1, 64, 64])
item  182  image shape=  torch.Size([1, 64, 64])
item  183  image shape=  torch.Size([1, 64, 64])
item  184  image shape=  torch.Size([1, 64, 64])
item  185  image shape=  torch.Size([1, 64, 64])
item  186  image shape=  torch.Size([1, 64, 64])
item  187  image shape=  torch.Size([1, 64, 64])
item  188  image shape=  torch.Size([1, 64, 64])
item  189  image shape=  torch.Size([1, 64, 64])
item  190  image shape=  torch.Size([1, 64, 64])
item  191  image shape=  torch.Size([1, 64, 64])
item  192  image shape=  torch.Size([1, 64, 64])
item  193  image shape=  torch.Size([1, 64, 64])
item  194  image shape=  torch.Size([1, 64, 64])
item  195  image sha

item  352  image shape=  torch.Size([1, 64, 64])
item  353  image shape=  torch.Size([1, 64, 64])
item  354  image shape=  torch.Size([1, 64, 64])
item  355  image shape=  torch.Size([1, 64, 64])
item  356  image shape=  torch.Size([1, 64, 64])
item  357  image shape=  torch.Size([1, 64, 64])
item  358  image shape=  torch.Size([1, 64, 64])
item  359  image shape=  torch.Size([1, 64, 64])
item  360  image shape=  torch.Size([1, 64, 64])
item  361  image shape=  torch.Size([1, 64, 64])
item  362  image shape=  torch.Size([1, 64, 64])
item  363  image shape=  torch.Size([1, 64, 64])
item  364  image shape=  torch.Size([1, 64, 64])
item  365  image shape=  torch.Size([1, 64, 64])
item  366  image shape=  torch.Size([1, 64, 64])
item  367  image shape=  torch.Size([1, 64, 64])
item  368  image shape=  torch.Size([1, 64, 64])
item  369  image shape=  torch.Size([1, 64, 64])
item  370  image shape=  torch.Size([1, 64, 64])
item  371  image shape=  torch.Size([1, 64, 64])
item  372  image sha

item  529  image shape=  torch.Size([1, 64, 64])
item  530  image shape=  torch.Size([1, 64, 64])
item  531  image shape=  torch.Size([1, 64, 64])
item  532  image shape=  torch.Size([1, 64, 64])
item  533  image shape=  torch.Size([1, 64, 64])
item  534  image shape=  torch.Size([1, 64, 64])
item  535  image shape=  torch.Size([1, 64, 64])
item  536  image shape=  torch.Size([1, 64, 64])
item  537  image shape=  torch.Size([1, 64, 64])
item  538  image shape=  torch.Size([1, 64, 64])
item  539  image shape=  torch.Size([1, 64, 64])
item  540  image shape=  torch.Size([1, 64, 64])
item  541  image shape=  torch.Size([1, 64, 64])
item  542  image shape=  torch.Size([1, 64, 64])
item  543  image shape=  torch.Size([1, 64, 64])
item  544  image shape=  torch.Size([1, 64, 64])
item  545  image shape=  torch.Size([1, 64, 64])
item  546  image shape=  torch.Size([1, 64, 64])
item  547  image shape=  torch.Size([1, 64, 64])
item  548  image shape=  torch.Size([1, 64, 64])
item  549  image sha

item  706  image shape=  torch.Size([1, 64, 64])
item  707  image shape=  torch.Size([1, 64, 64])
item  708  image shape=  torch.Size([1, 64, 64])
item  709  image shape=  torch.Size([1, 64, 64])
item  710  image shape=  torch.Size([1, 64, 64])
item  711  image shape=  torch.Size([1, 64, 64])
item  712  image shape=  torch.Size([1, 64, 64])
item  713  image shape=  torch.Size([1, 64, 64])
item  714  image shape=  torch.Size([1, 64, 64])
item  715  image shape=  torch.Size([1, 64, 64])
item  716  image shape=  torch.Size([1, 64, 64])
item  717  image shape=  torch.Size([1, 64, 64])
item  718  image shape=  torch.Size([1, 64, 64])
item  719  image shape=  torch.Size([1, 64, 64])
item  720  image shape=  torch.Size([1, 64, 64])
item  721  image shape=  torch.Size([1, 64, 64])
item  722  image shape=  torch.Size([1, 64, 64])
item  723  image shape=  torch.Size([1, 64, 64])
item  724  image shape=  torch.Size([1, 64, 64])
item  725  image shape=  torch.Size([1, 64, 64])
item  726  image sha

...

item  2099  image shape=  torch.Size([1, 64, 64])
item  2100  image shape=  torch.Size([1, 64, 64])
item  2101  image shape=  torch.Size([1, 64, 64])
item  2102  image shape=  torch.Size([1, 64, 64])
item  2103  image shape=  torch.Size([1, 64, 64])
item  2104  image shape=  torch.Size([1, 64, 64])
item  2105  image shape=  torch.Size([1, 64, 64])
item  2106  image shape=  torch.Size([1, 64, 64])
item  2107  image shape=  torch.Size([1, 64, 64])
item  2108  image shape=  torch.Size([1, 64, 64])
item  2109  image shape=  torch.Size([1, 64, 64])
item  2110  image shape=  torch.Size([1, 64, 64])
item  2111  image shape=  torch.Size([1, 64, 64])
item  2112  image shape=  torch.Size([1, 64, 64])
item  2113  image shape=  torch.Size([1, 64, 64])
item  2114  image shape=  torch.Size([1, 64, 64])
item  2115  image shape=  torch.Size([1, 64, 64])
item  2116  image shape=  torch.Size([1, 64, 64])
item  2117  image shape=  torch.Size([1, 64, 64])
item  2118  image shape=  torch.Size([1, 64, 64])


item  2275  image shape=  torch.Size([1, 64, 64])
item  2276  image shape=  torch.Size([1, 64, 64])
item  2277  image shape=  torch.Size([1, 64, 64])
item  2278  image shape=  torch.Size([1, 64, 64])
item  2279  image shape=  torch.Size([1, 64, 64])
item  2280  image shape=  torch.Size([1, 64, 64])
item  2281  image shape=  torch.Size([1, 64, 64])
item  2282  image shape=  torch.Size([1, 64, 64])
item  2283  image shape=  torch.Size([1, 64, 64])
item  2284  image shape=  torch.Size([1, 64, 64])
item  2285  image shape=  torch.Size([1, 64, 64])
item  2286  image shape=  torch.Size([1, 64, 64])
item  2287  image shape=  torch.Size([1, 64, 64])
item  2288  image shape=  torch.Size([1, 64, 64])
item  2289  image shape=  torch.Size([1, 64, 64])
item  2290  image shape=  torch.Size([1, 64, 64])
item  2291  image shape=  torch.Size([1, 64, 64])
item  2292  image shape=  torch.Size([1, 64, 64])
item  2293  image shape=  torch.Size([1, 64, 64])
item  2294  image shape=  torch.Size([1, 64, 64])


item  2482  image shape=  torch.Size([1, 64, 64])
item  2483  image shape=  torch.Size([1, 64, 64])
item  2484  image shape=  torch.Size([1, 64, 64])
item  2485  image shape=  torch.Size([1, 64, 64])
item  2486  image shape=  torch.Size([1, 64, 64])
item  2487  image shape=  torch.Size([1, 64, 64])
item  2488  image shape=  torch.Size([1, 64, 64])
item  2489  image shape=  torch.Size([1, 64, 64])
item  2490  image shape=  torch.Size([1, 64, 64])
item  2491  image shape=  torch.Size([1, 64, 64])
item  2492  image shape=  torch.Size([1, 64, 64])
item  2493  image shape=  torch.Size([1, 64, 64])
item  2494  image shape=  torch.Size([1, 64, 64])
item  2495  image shape=  torch.Size([1, 64, 64])
item  2496  image shape=  torch.Size([1, 64, 64])
item  2497  image shape=  torch.Size([1, 64, 64])
item  2498  image shape=  torch.Size([1, 64, 64])
item  2499  image shape=  torch.Size([1, 64, 64])
item  2500  image shape=  torch.Size([1, 64, 64])
item  2501  image shape=  torch.Size([1, 64, 64])


item  2704  image shape=  torch.Size([1, 64, 64])
item  2705  image shape=  torch.Size([1, 64, 64])
item  2706  image shape=  torch.Size([1, 64, 64])
item  2707  image shape=  torch.Size([1, 64, 64])
item  2708  image shape=  torch.Size([1, 64, 64])
item  2709  image shape=  torch.Size([1, 64, 64])
item  2710  image shape=  torch.Size([1, 64, 64])
item  2711  image shape=  torch.Size([1, 64, 64])
item  2712  image shape=  torch.Size([1, 64, 64])
item  2713  image shape=  torch.Size([1, 64, 64])
item  2714  image shape=  torch.Size([1, 64, 64])
item  2715  image shape=  torch.Size([1, 64, 64])
item  2716  image shape=  torch.Size([1, 64, 64])
item  2717  image shape=  torch.Size([1, 64, 64])
item  2718  image shape=  torch.Size([1, 64, 64])
item  2719  image shape=  torch.Size([1, 64, 64])
item  2720  image shape=  torch.Size([1, 64, 64])
item  2721  image shape=  torch.Size([1, 64, 64])
item  2722  image shape=  torch.Size([1, 64, 64])
item  2723  image shape=  torch.Size([1, 64, 64])


item  2891  image shape=  torch.Size([1, 64, 64])
item  2892  image shape=  torch.Size([1, 64, 64])
item  2893  image shape=  torch.Size([1, 64, 64])
item  2894  image shape=  torch.Size([1, 64, 64])
item  2895  image shape=  torch.Size([1, 64, 64])
item  2896  image shape=  torch.Size([1, 64, 64])
item  2897  image shape=  torch.Size([1, 64, 64])
item  2898  image shape=  torch.Size([1, 64, 64])
item  2899  image shape=  torch.Size([1, 64, 64])
item  2900  image shape=  torch.Size([1, 64, 64])
item  2901  image shape=  torch.Size([1, 64, 64])
item  2902  image shape=  torch.Size([1, 64, 64])
item  2903  image shape=  torch.Size([1, 64, 64])
item  2904  image shape=  torch.Size([1, 64, 64])
item  2905  image shape=  torch.Size([1, 64, 64])
item  2906  image shape=  torch.Size([1, 64, 64])
item  2907  image shape=  torch.Size([1, 64, 64])
item  2908  image shape=  torch.Size([1, 64, 64])
item  2909  image shape=  torch.Size([1, 64, 64])
item  2910  image shape=  torch.Size([1, 64, 64])


In [6]:
dset = getActualImgs(dataset, [0,1,2], 1)
print(type(dset))

<class 'torch.utils.data.dataloader.DataLoader'>


In [7]:
for batch_idx, (data, target) in enumerate(dset):
    print(data)

tensor([[[[0, 0, 0,  ..., 0, 0, 4],
          [0, 0, 0,  ..., 0, 0, 1],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]]], dtype=torch.uint8)
tensor([[[[57, 55, 52,  ..., 20, 32, 47],
          [81, 78, 72,  ..., 48, 61, 73],
          [93, 86, 83,  ..., 68, 83, 98],
          ...,
          [ 0,  0, 30,  ..., 13,  0,  0],
          [ 0,  0, 30,  ..., 11,  0,  0],
          [ 0,  0, 29,  ..., 10,  0,  0]]]], dtype=torch.uint8)
tensor([[[[ 0,  0,  3,  ..., 35, 55, 75],
          [ 0,  0,  1,  ..., 58, 70, 79],
          [ 0,  0,  0,  ..., 70, 82, 82],
          ...,
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0],
          [ 0,  0,  0,  ...,  0,  0,  0]]]], dtype=torch.uint8)


/home/citi302/Desktop/Codefolder/FL_DP_covid/FLDataset.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(images).clone().detach(), torch.tensor(label).clone().detach()
